###  아래 가이드에 따라서 1000명의 고객 구매데이터를 생성하고 아래사항을 수행하세요(관측치 10000개)

In [1]:
import pandas as pd
import numpy as np
import random
from datetime import datetime
import time
import matplotlib.pyplot as plt
import seaborn as sns

#### Q1. 고객 속성(id, 성별, 연령, 거주지역) 데이터와 거래 데이터를 통합한 데이터 프레임 df를 생성하세요.

In [8]:
# 고객 데이터프레임
np.random.seed(0)
cusno = np.arange(1001,2001)
gender = np.random.randint(0,2,size=1000)
age = np.random.randint(10,81,size=1000)
region = np.random.randint(1,11,size=1000)

cus = pd.DataFrame({'cusno':cusno,'gender':gender,'age':age,'region':region})

# 거래 데이터프레임
trans_id = random.sample(list(range(10001,50000)), 10000)
cusno = np.random.randint(1001,2001, size=10000)
date = pd.date_range(start='2020-01-01', end='2021-12-31', periods=10000, normalize=True)
day = date.day_name
product_20 = np.random.choice([10,20,30], 10000) # 10:고가, 20:중가, 30:저가
product_21 = np.random.choice([10,20,30], 10000)
price_20 = np.random.uniform(1000,50000,10000).round()
price_21 = np.random.uniform(1000,50000,10000).round()
quantity_20 = np.random.randint(1,101, size=10000)
quantity_21 = np.random.randint(1,101, size=10000)
time_20 = np.random.randint(1,25, size=10000)
time_21 = np.random.randint(1,25, size=10000)
amount_20 = price_20 * quantity_21
amount_21 = price_21 * quantity_21

trans = pd.DataFrame({
    'trans_id':trans_id, 'cusno':cusno, 'date':date, 'day':day(), 'product_20':product_20,
    'product_21':product_21, 'price_20':price_20, 'price_21':price_21,
    'quantity_20':quantity_20, 'quantity_21':quantity_21,
    'time_20':time_20, 'time_21':time_21, 'amount_20':amount_20, 'amount_21':amount_21})

# cus와 trans merge
df = pd.merge(cus, trans, on='cusno', how='outer')


#### Q2. df를 수정하여 5가지 이상의 인사이트를 포함한 데이터 셋으로 변환하세요

In [ ]:
# 인사이트 삽입

# 1. 연령대별 월, 분기, 연평균 매출
# 2. 연령대별 상품 선호도
#   - 10~20대 : 중저가 선호, 30대~50대 : 중고가 선호, 60~70대 : 고가 선호

# 2. 분기별 매출 변동 : 상품별 변동률 산출
#   - 1분기 1~3월   : 중저가 매출 상승 : 개학
#   - 2분기 4~6월   : 중고가 매출 상승 : 결혼시즌
#   - 3분기 7~9월   : 고가 매출 상승   : 에어컨 시즌
#   - 4분기 10~12월 : 저가 매출 상승   : 연말

# 3. 2020년 대비 2021년 상품 평균 가격 20% 상승
#   - 연 평균 매출은 25% 하락

# 4. 지역별 월, 분기, 연평균 매출

# 5. 지역별 연도별 상품 선호도 변화

In [ ]:
# 월, 분기, 연도별 데이터셋
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter

# price값에 따라 product 값 변경 필요
# 1000원 ~ 50000원이니까 1000원~15000원 저가, 15001원~35000원 중가, 35001원~50000원 고가 설정

def price_readjust(x):
  if 1000 <= x <= 15000:
    return 10
  elif 15000 < x <= 35000:
    return 20
  else:
    return 30

trans['product_20'] = trans['price_20'].apply(price_readjust)
trans['product_21'] = trans['price_21'].apply(price_readjust)

# 연령대 카테고리 생성
bins = [10, 19, 29, 39, 49, 59, 69, 80]
labels = ['10s', '20s', '30s', '40s', '50s', '60s', '70s']
df['age_group'] = pd.cut(df['age'], bins=bins, labels=labels, right=False)

In [ ]:
# 고객 연령 분포 확인
plt.figure(figsize=(10, 6))
sns.histplot(cus['age'], bins=20, kde=True, color='skyblue')
plt.title('Age Distribution of Customers')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# 월별, 분기별, 연도별 매출 현황 체크
monthly_sales = pd.pivot_table(df, values='amount_20', index='month', columns='year', aggfunc='sum')
quarterly_sales = pd.pivot_table(df, values='amount_21', index='quarter', columns='year', aggfunc='sum')
annual_sales = pd.pivot_table(df, values='amount_21', index='year', aggfunc='sum')

monthly_sales.plot(kind='bar', figsize=(10, 5))
plt.title('Monthly Sales Trends')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.legend(title='Year')
plt.show()

In [ ]:
# 제품 선호도 조사
# 연령대별, 지역별

pivot_table = pd.pivot_table(df, values=['quantity_20', 'quantity_21'], index='region', columns=['product_20', 'product_21'], aggfunc='sum')
print(pivot_table)


#### Q3. df 를 탐색적 분석을 통하여 인사이트를 도출하세요.